In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split


wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state= 42, test_size= 0.2)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state= 42, test_size= 0.2)

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [13]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target, cv=5)
scores

{'fit_time': array([0.00799108, 0.00800061, 0.00800347, 0.00800848, 0.00798988]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [14]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [15]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle= True, random_state= 42)
scores = cross_validate(dt, train_input, train_target, cv= splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [23]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' : [0.001,0.002,0.003,0.004,0.005]
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
gs.cv_results_['mean_test_score']

array([0.86165044, 0.85761105, 0.85607111, 0.85414803, 0.85318557])

In [28]:
params = {
    'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
    'max_depth' : np.arange(5,20,1),
    'min_samples_split' : np.arange(2,100,10)
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs= -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': array([ 2, 12, 22, 32, 42, 52, 62, 72, 82, 92])})

In [31]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.892053107562055

In [55]:
from scipy.stats import uniform, randint

params = {
    'min_impurity_decrease' : uniform(0.0001,0.001),
    'max_depth' : randint(20,50),
    'min_samples_split' : randint(2,25),
    'min_samples_leaf' : randint(1,25)
}
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, n_iter=1000)
gs.fit(train_input, train_target)
dt = gs.best_estimator_

print(dt.score(train_input, train_target), dt.score(test_input,test_target))

0.8907061766403694 0.8576923076923076


In [50]:
dt = gs.best_estimator_
dt.score(test_input,test_target)

0.8584615384615385